# Mongo desde Python 


Para una introducción al uso de Mongo desde Python, ver:

[http://api.mongodb.com/python/current/tutorial.html]

Aquí solo vamos a ver unos pocos ejemplos con dos ficheros json de comentarios de un foro. El primer fichero, usuarios.json, tiene los usuarios, que tienen un identificador y un *nick*. El segundo tiene los comentarios. Nos gustaría añadir el nick a cada comentario, sabiendo que el campo "content.authorId" ya contiene el identificador, lo que nos permitirá cruzar ambos ficheros.

Ya en Python, empezamos por importar la librería que permite cargar datos desde mongo

In [1]:
import pymongo  # la conexión con mongo
from pprint import pprint

In [2]:
from pymongo import MongoClient
# cliente = MongoClient('mongodb://127.0.0.0:27017');
cliente = MongoClient();



Empezamos por acceder a la base de datos. Lo hacemos a través del cliente.

In [3]:
# db = client.pruebas
db = cliente.test;

Un insert de prueba

In [4]:
import datetime
queja = {"name": "Herminia",
         "level": 5,
         "tags": ["mongodb", "python", "pymongo"],
         "date": datetime.datetime.utcnow(),
         "comment": "Mongo casca con error 121"}

db.quejas.drop() # Ojo, esto borra completamente la colección
db.quejas.insert_one(queja)

Comprobamos que se ha creado listando las colecciones en la BD

In [5]:
colecciones = db.list_collection_names()
print(colecciones)

['quejas', 'Empleados', 'compras']


Veamos cuantos elementos tiene cada coleccion

In [6]:
for col in colecciones:
    print(col,db[col].count_documents({}))

quejas 1
Empleados 40
compras 1000


In [7]:
unaqueja = db.quejas.find_one() # el primer usuario
pprint(unaqueja)

{'_id': ObjectId('5e3307409ac07687a578cdc4'),
 'comment': 'Mongo casca con error 121',
 'date': datetime.datetime(2020, 1, 30, 16, 41, 36, 571000),
 'level': 5,
 'name': 'Herminia',
 'tags': ['mongodb', 'python', 'pymongo']}


Vamos a crear una colección con datos aleatorios

In [8]:
import random
import datetime
nombres = ['bertoldo','herminia','aniceto','calixta','melibea']

total = 1000 # total de usuarios a añadir

l = []
db.compras.drop()
for i in range(total):
    nombre= random.choice(nombres)+"_"+str(i)    
    edad = random.randint(1,101)
    totalcompras = random.randint(0,21)
    compras = []
    for j in range(totalcompras):
        compras.append(random.randint(1,200))
    dato = {"_id":i,"nombre": nombre, "edad":edad,"precios":compras, 
             "dir":{"calle":"rosa "+str(i %10), "num":str(i+len(compras)%50)},
             "hora":datetime.datetime.utcnow()}
    l.append(dato)
db.compras.insert_many(l)
print("Insertados ",db.compras.count_documents({}), " documentos")


Insertados  1000  documentos


Para que vaya más rápido podemos crear un índice

In [9]:
db.compras.drop_indexes()
result = db.compras.create_index([('precios', pymongo.ASCENDING)],
                                   unique=False)
print(result)

precios_1


Estamos interesados en saber los nombres de los menores que han hecho alguna compra

In [10]:
query = db.compras.find({"edad":{"$lt":18}})
for doc in query:
    print(doc['nombre'],doc['edad'])

bertoldo_5 14
bertoldo_14 12
calixta_19 3
herminia_23 9
calixta_27 4
bertoldo_34 10
bertoldo_35 15
calixta_36 3
melibea_37 1
bertoldo_41 16
calixta_43 11
bertoldo_50 11
melibea_58 6
aniceto_60 17
melibea_88 9
aniceto_89 13
melibea_92 16
aniceto_97 5
melibea_102 11
melibea_107 9
calixta_109 1
herminia_110 17
melibea_114 12
melibea_120 10
bertoldo_127 8
melibea_131 10
herminia_137 11
melibea_141 5
calixta_143 12
herminia_146 6
herminia_171 16
herminia_172 7
calixta_182 16
aniceto_186 7
aniceto_191 8
herminia_213 11
bertoldo_214 11
bertoldo_221 8
aniceto_224 11
calixta_225 4
herminia_227 8
herminia_229 8
calixta_230 17
melibea_242 11
aniceto_249 1
calixta_274 1
herminia_276 12
herminia_278 14
melibea_285 13
bertoldo_286 15
aniceto_290 17
calixta_297 7
aniceto_306 8
herminia_309 15
calixta_317 14
aniceto_327 3
aniceto_334 12
herminia_335 12
calixta_357 8
herminia_385 8
bertoldo_389 13
herminia_405 9
bertoldo_410 17
aniceto_416 10
herminia_424 12
herminia_430 7
aniceto_431 5
bertoldo_434 8


Queremos saber los nombres de los que hayan hecho alguna compra entre 180 y 190 euros.
En caso de que haya varios elementos de este tipo nos basta con mostrar el primero

```
{'nombre': 'aniceto_3', 'precios': [185]}
{'nombre': 'herminia_6', 'precios': [183]}
{'nombre': 'melibea_7', 'precios': [181]}
{'nombre': 'aniceto_9', 'precios': [190]}
```

In [11]:
query = db.compras.find({"precios":{"$elemMatch":{"$gte":180,"$lte":190}}}) # ,{"nombre":1,"precios.$":1,"_id":0}
for doc in query:
    print(doc)

{'_id': 2, 'nombre': 'calixta_2', 'edad': 67, 'precios': [183, 74, 178, 63, 180, 162, 171, 33, 82, 75, 140, 142, 51], 'dir': {'calle': 'rosa 2', 'num': '15'}, 'hora': datetime.datetime(2020, 1, 30, 16, 41, 36, 607000)}
{'_id': 14, 'nombre': 'bertoldo_14', 'edad': 12, 'precios': [110, 104, 23, 65, 135, 180, 186, 105, 13, 180, 60, 126, 138, 139, 139, 47, 133, 8, 11, 72], 'dir': {'calle': 'rosa 4', 'num': '34'}, 'hora': datetime.datetime(2020, 1, 30, 16, 41, 36, 607000)}
{'_id': 21, 'nombre': 'herminia_21', 'edad': 36, 'precios': [6, 2, 86, 180, 189, 186, 171, 141, 33], 'dir': {'calle': 'rosa 1', 'num': '30'}, 'hora': datetime.datetime(2020, 1, 30, 16, 41, 36, 607000)}
{'_id': 22, 'nombre': 'melibea_22', 'edad': 35, 'precios': [68, 67, 23, 3, 69, 31, 126, 174, 177, 110, 180, 175, 175, 194, 93, 106, 58, 91], 'dir': {'calle': 'rosa 2', 'num': '40'}, 'hora': datetime.datetime(2020, 1, 30, 16, 41, 36, 607000)}
{'_id': 48, 'nombre': 'calixta_48', 'edad': 88, 'precios': [8, 128, 180, 112, 52, 1

Obtener los documentos de personas que se llamen herminia_XXX y que el número de calle sea menor que 50.

In [24]:
query = db.compras.find({ "nombre": {"$regex": "^herminia_"} , "$where": "function() { return Number(this.dir.num) > 50; }" },{ "nombre": 1 , "dir.num": 1 , "_id": 0 });
for doc in query:
    print(doc)

{'nombre': 'herminia_32', 'dir': {'num': '53'}}
{'nombre': 'herminia_40', 'dir': {'num': '53'}}
{'nombre': 'herminia_56', 'dir': {'num': '62'}}
{'nombre': 'herminia_65', 'dir': {'num': '69'}}
{'nombre': 'herminia_66', 'dir': {'num': '76'}}
{'nombre': 'herminia_75', 'dir': {'num': '91'}}
{'nombre': 'herminia_87', 'dir': {'num': '106'}}
{'nombre': 'herminia_93', 'dir': {'num': '100'}}
{'nombre': 'herminia_99', 'dir': {'num': '104'}}
{'nombre': 'herminia_101', 'dir': {'num': '113'}}
{'nombre': 'herminia_103', 'dir': {'num': '113'}}
{'nombre': 'herminia_105', 'dir': {'num': '114'}}
{'nombre': 'herminia_110', 'dir': {'num': '115'}}
{'nombre': 'herminia_116', 'dir': {'num': '129'}}
{'nombre': 'herminia_119', 'dir': {'num': '127'}}
{'nombre': 'herminia_124', 'dir': {'num': '132'}}
{'nombre': 'herminia_134', 'dir': {'num': '136'}}
{'nombre': 'herminia_137', 'dir': {'num': '155'}}
{'nombre': 'herminia_138', 'dir': {'num': '153'}}
{'nombre': 'herminia_140', 'dir': {'num': '141'}}
{'nombre': 'her